# Trainingspipeline 0

- ohne Bigramme
- vector_size: 300
- window: 10
- seed: 42

In [1]:
import codecs
import nltk
import numpy as np
import os
import pandas as pd
import re
import scipy
import spacy

#from gensim.models.phrases import Phraser, Phrases
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from joblib import Parallel, delayed  
from nltk.corpus import stopwords
from tabulate import tabulate

In [2]:
stopwords = stopwords.words('italian')
tokenizer = nltk.data.load('tokenizers/punkt/italian.pickle')

In [3]:
df = pd.read_csv('../Korpus/Korpus/corpus_final.csv', sep=',', encoding='utf-8')

In [4]:
df.head()

,doc,source,author,title,year,period,text_type,text,words,lemmatized_text,cleaned_tokenized_text
0,Poesia.IV.4.Testo.txt,MIDIA,Faustina Maratti Zappi,Poesie,1700.0,1700-1750,poesia,IV. 4. Rime degli Arcadi: Aglauro Cidonia (Fau...,3184.0,iv . 4 . rima del arcadi : aglauro cidonia ( f...,"[['iv'], [], ['rima', 'arcadi', 'aglauro', 'ci..."
1,Espositivi.IV.4.Testo.txt,MIDIA,Ludovico Antonio Muratori,Antichità italiane,1700.0,1700-1750,espositivo,"﻿IV. 4. Ludovico Antonio Muratori, Antichità i...",8990.0,"﻿iv . 4 . Ludovico Antonio muratori , antichit...","[['iv'], [], ['ludovico', 'antonio', 'muratori..."
2,Personali.IV.5.Testo.txt,MIDIA,Lorenzo Magalotti,Lettere odorose (1693-1705),1700.0,1700-1750,personale,"IV. 5. Lorenzo Magalotti, Lettere odorose (169...",8374.0,"iv . 5 . Lorenzo magalotti , lettere odoroso (...","[['iv'], [], ['lorenzo', 'magalotti', 'lettere..."
3,Personali.IV.15.Testo.txt,MIDIA,Pietro Giannone,Vita scritta da lui medesimo,1700.0,1700-1750,personale,[Proemio]\nPrendo a scrivere la mia vita e qua...,10118.0,[ proemio ] \n prendere a scrivere il mio vita...,"[['proemio', 'prendere', 'scrivere', 'vita', '..."
4,Personali.IV.4.Testo.txt,MIDIA,Vincenzo da Filicaia,Lettere inedite a Lorenzo Magalotti,1700.0,1700-1750,personale,"IV. 4. Vincenzo da Filicaia, Lettere inedite a...",10073.0,"iv . 4 . Vincenzo da filicaia , lettere inedit...","[['iv'], [], ['vincenzo', 'filicaia', 'lettere..."


In [5]:
df.shape

(710840, 11)

In [6]:
df.text = df.text.fillna('')
df.lemmatized_text = df.lemmatized_text.fillna('')

In [7]:
# Einzeldataframes für die Zeiträume

df_periods = dict(tuple(df.groupby(by='period')))

In [8]:
df1 = df_periods['1700-1750']
df2 = df_periods['1751-1800']
df3 = df_periods['1801-1825']
df4 = df_periods['1826-1850']
df5 = df_periods['1851-1875']
df6 = df_periods['1876-1900']
df7 = df_periods['1901-1925']
df8 = df_periods['1926-1950']
df9 = df_periods['1951-1975']
df10 = df_periods['1976-2000']
df11 = df_periods['2001-2010']
df12 = df_periods['2011-2016']
df13 = df_periods['2017-2021']

In [9]:
df9.head()

,doc,source,author,title,year,period,text_type,text,words,lemmatized_text,cleaned_tokenized_text
94810,LISRodari3.txt,LIS,Gianno Rodari,La questione dei fumetti,1951.0,1951-1975,stampa,"Caro Direttore , ho letto nell ' ultimo numero...",1510.0,"caro direttore , avere leggere nell ' ultimo n...","[['caro', 'direttore', 'avere', 'leggere', 'ul..."
94811,LISJotti1.txt,LIS,Nilde Jotti,La questione dei fumetti,1951.0,1951-1975,stampa,Il dibattito sulla stampa a fumetti per i raga...,2785.0,il dibattito sulla stampa a fumetto per il rag...,"[['dibattito', 'stampa', 'fumetto', 'ragazzo',..."
94812,LLAlbertelli1.txt,Liber Liber,Pilo Albertelli,Rousseau,1951.0,1951-1975,prosa letteraria,﻿Pilo Albertelli\nRousseau\n\n Nacque il 28 g...,4894.0,﻿pilo albertelli \n rousseau \n\n nascere il...,"[['pilo', 'albertelli', 'rousseau', 'nascere',..."
94813,LISManacorda1.txt,LIS,Gastone Manacorda,Il Partito e la sua funzione di guida nel camp...,1951.0,1951-1975,stampa,"Al partito , nel suo rigoglioso sviluppo , seg...",3460.0,"al partito , nel suo rigoglioso sviluppo , seg...","[['partito', 'rigoglioso', 'sviluppo', 'seguit..."
94814,LISBianchi1.txt,LIS,Ranuccio Bianchi Bandinelli,Il nostro lavoro nella scuola,1951.0,1951-1975,stampa,"Come in tutti i congressi , anche nel VII Cong...",2898.0,"come in tutto il congresso , anche nel vii con...","[['congresso', 'vii', 'congresso', 'p'], ['tes..."


## Training von Word2Vec

In [19]:
# Hilfsfunktionen zur Vorbereitung auf das Training
# Bereinigung und Tokenisierung

def sentence_to_wordlist(raw:str):
    """
    cleans and tokenizes the sentences
    """
    text = re.sub('[^A-Za-z_àÀèÈìÌòÒùÙáÁéÉíÍóÓúÚ]',' ', raw).split()        # Diakritika ans Italienische anpassen                    
    filtered_text = [word for word in text if word not in stopwords]        # Stopwörter löschen
    return filtered_text


def tokenize_text(raw_text):
    """
    returns a list of lowercase tokenized sentences 
    """
    raw_sentences = tokenizer.tokenize(str(raw_text).lower())    
    tokenized_sentences = Parallel(n_jobs=-1)(delayed(sentence_to_wordlist)(raw_sentence) for raw_sentence in raw_sentences)
    sentences = tokenized_sentences
    return sentences

In [20]:
# Trainingsparamter setzen

vector_size = 300                  # Dimensionality of the word vectors
window = 10                        # The maximum distance between the current and predicted word within a sentence
min_count = 2                      # (int, optional) – The model ignores all words with total frequency lower than this
workers = 1                        # Use these many worker threads to train the model (faster training with multicore machines)
min_alpha = 0.0001                 # Learning rate will linearly drop to min_alpha as training progresses
sg = 1                             # Training algorithm: skip-gram if sg=1, otherwise CBOW            
seed = 42                          # Reproductivity (42 just because...) --> only if workers = 1

In [21]:
# Ordner anlegen zum Abspeichern von trainierten Modellen

if not os.path.exists('../trained_models'):
    os.makedirs('../trained_models')

### Zeitraum 1: 1700-1750

In [22]:
# lemmatisierte Texte zu einem String verbinden

text1 = ''

for i in df1.lemmatized_text:
    text1 += i

In [23]:
%%time
sentences1 = tokenize_text(text1)         # Bereinigen, Tokenisieren und in Form bringen (Ziel: Liste von tokenisierten Sätzen)

Wall time: 11.3 s


In [25]:
print(sentences1[:5])

[['iv'], [], ['rima', 'arcadi', 'aglauro', 'cidonia', 'faustina', 'maratti', 'zappi', 'aglauro', 'cidonia', 'credea', 'debil', 'navicella', 'rotta', 'onda', 'stancare', 'cammino', 'ritrar', 'porto', 'scorgea', 'vicino', 'ch', 'troppo', 'scorso', 'parte'], ['credea', 'gi', 'calmare', 'ogni', 'procella', 'saziare', 'parte', 'crudel', 'destino', 'ciel', 'pi', 'sereno', 'me', 'divino', 'raggio', 'mostrare', 'propizio', 'stella'], ['barbaro', 'clima', 'vento', 'sorgere', 'sospingere', 'forza', 'scoglio', 'tal', 'naviglio', 'ahi', 'fia', 'onda', 'assorto']]


In [26]:
len(sentences1)

82675

In [27]:
%%time

# Training   

w2v1 = Word2Vec(sentences=sentences1,                      
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 54s


In [28]:
w2v1.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.7818655371665955),
 ('armi', 0.7175211906433105),
 ('goti', 0.7045249342918396),
 ('paura', 0.6868568658828735),
 ('faceano', 0.6861615777015686),
 ('spaventare', 0.6838738322257996),
 ('barbari', 0.6772375702857971),
 ('costernazione', 0.6576120853424072),
 ('addossare', 0.6540018320083618),
 ('pannonia', 0.6525850892066956)]

In [29]:
# trainiertes Modell speichern

w2v1.save(os.path.join('../trained_models/Word2Vec0', 'w2v1.model'))

### Zeitraum 2: 1751-1800

In [30]:
text2 = ''

for i in df2.lemmatized_text:
    text2 += i

In [31]:
%%time
sentences2 = tokenize_text(text2)

Wall time: 16.1 s


In [32]:
print(sentences2[:5])

[['vita', 'antonio', 'genovese', 'antonio', 'genovese', 'avo', 'avere', 'ereditare', 'padre', 'dote', 'moglie', 'giustina', 'genovese', 'avere', 'terra', 'tanto', 'onda', 'poco', 'industria', 'avere', 'potere', 'solo', 'vivere', 'comodit', 'molto', 'accrescere', 'patrimonio'], ['poltroneria', 'scem'], ['avere', 'sette', 'figlio', 'tre', 'maschio', 'quattro', 'femine'], ['femine', 'due', 'morire', 'pulcelle', 'due', 'essere', 'maritate'], ['de', 'maschio', 'secondo', 'mor', 'cherico', 'ultimo', 'cambio', 'cielo', 'stabil', 'altrove']]


In [33]:
len(sentences2)

144740

In [34]:
%%time
w2v2 = Word2Vec(sentences=sentences2,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 8s


In [35]:
w2v2.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.7384335994720459),
 ('ostile', 0.6631743311882019),
 ('barbari', 0.6546300053596497),
 ('legione', 0.6460967063903809),
 ('inspirare', 0.6396669149398804),
 ('orrore', 0.636448085308075),
 ('atterrire', 0.6330591440200806),
 ('strage', 0.6319993138313293),
 ('usurpatore', 0.6258938312530518),
 ('crociati', 0.6238396167755127)]

In [36]:
w2v2.save(os.path.join('../trained_models/Word2Vec0', 'w2v2.model'))

### Zeitraum 3: 1801-1825

In [37]:
text3 = ''

for i in df3.lemmatized_text:
    text3 += i

In [38]:
%%time
sentences3 = tokenize_text(text3)

Wall time: 11.9 s


In [39]:
print(sentences3[:5])

[['effetti', 'fame', 'disperazione', 'uomo'], ['autore', 'viaggio', 'alto', 'pensilvania', 'nuovo', 'york', 'libro', 'stampare', 'poco', 'anno', 'filadelfia', 'ristampare', 'londra', 'descrivere', 'secondo', 'que', 'tre', 'volume', 'avvenimento', 'essere', 'riescire', 'funesto', 'tanto', 'compagno', 'viaggio', 'raccontare', 'tradurre', 'fedelmente', 'solo', 'perchè', 'opera', 'inglese', 'essere', 'poco', 'noto', 'italia', 'eziandio', 'perchè', 'fatto', 'confermare', 'alcune', 'poco', 'vero', 'nozione', 'tanto', 'secolo', 'esperienza', 'studio', 'avere', 'appena', 'potere', 'dare', 'natura', 'uomo'], ['compiacere', 'desiderio', 'esaminare', 'monte', 'allegheni', 'sig'], ['hermann', 'amicizia', 'abbandonare', 'mai', 'tanto', 'anno', 'viaggio', 'così', 'penoso', 'avviare', 'meco', 'fare', 'giorno', 'provvedendosi', 'battifuoco', 'pietra', 'focaia', 'bisogno', 'onda', 'scoprire', 'albero', 'ape', 'chiamare', 'paese', 'bee', 'tree'], ['ape', 'recare', 'primo', 'volta', 'europeo', 'crescere'

In [40]:
len(sentences3)

85065

In [41]:
%%time
w2v3 = Word2Vec(sentences=sentences3,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 47s


In [42]:
w2v3.wv.most_similar(positive=['terrore'], topn=10)

[('repentino', 0.7741665840148926),
 ('spavento', 0.7676851153373718),
 ('eccitare', 0.7564024925231934),
 ('calma', 0.7475305795669556),
 ('abbattimento', 0.7370861172676086),
 ('sorpresa', 0.7312561273574829),
 ('orrore', 0.730105459690094),
 ('atterrire', 0.7290980815887451),
 ('infiammare', 0.7270755767822266),
 ('calmare', 0.7255777716636658)]

In [43]:
w2v3.save(os.path.join('../trained_models/Word2Vec0', 'w2v3.model'))

### Zeitraum 4: 1826-1850

In [44]:
text4 = ''

for i in df4.lemmatized_text:
    text4 += i

In [45]:
%%time
sentences4 = tokenize_text(text4)

Wall time: 14.7 s


In [46]:
print(sentences4[:5])

[['canto', 'cominciar', 'novello', 'canto', 'tenere', 'eliconie', 'cima', 'prego', 'vergine', 'dee', 'concilio', 'santo', 'stile', 'conduciate', 'rima', 'topo', 'rana', 'caso', 'acerbo', 'ira', 'segno', 'insolito', 'carmi', 'prendere', 'dire'], ['cetra', 'avere', 'man', 'carta', 'grembo', 'or', 'data', 'principio', 'fine', 'opra', 'virtù', 'molto', 'tardo', 'etate', 'suoni', 'dive', 'carme', 'fia', 'foglio', 'sacrati', 'scrivere', 'chiaro', 'fama', 'eternamente', 'vivo'], ['terrigeni', 'eroi', 'vasto', 'giganti', 'que', 'topo', 'imitare', 'schiatta', 'audace', 'dolor', 'furor', 'caldo', 'spumante', 'venire', 'campo', 'essere', 'fallace', 'memoria', 'romor', 'ch', 'oggi', 'restare', 'cagion', 'de', 'collera', 'essere'], ['topo', 'de', 'membra', 'molto', 'ben', 'fare', 'venne', 'lago', 'sponda', 'giorno'], ['campato', 'poco', 'innanzi', 'essere', 'gatto', 'ch', 'inseguire', 'avea', 'dintorno', 'stanco', 'faceasi', 'bere', 'quando', 'ranocchio', 'passare', 'vicin', 'porre', 'occhio']]


In [47]:
len(sentences4)

109750

In [48]:
%%time
w2v4 = Word2Vec(sentences=sentences4,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 27s


In [49]:
w2v4.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.6297602653503418),
 ('bentosto', 0.5547863245010376),
 ('atterrire', 0.5531451106071472),
 ('cagionare', 0.533476710319519),
 ('fuga', 0.5299801230430603),
 ('inspirare', 0.5289738178253174),
 ('buonaparte', 0.5227904319763184),
 ('scompiglio', 0.5160790681838989),
 ('repubblicano', 0.5114613175392151),
 ('panico', 0.5059127807617188)]

In [50]:
w2v4.save(os.path.join('../trained_models/Word2Vec0', 'w2v4.model'))

### Zeitraum 5: 1851-1875

In [51]:
text5 = ''

for i in df5.lemmatized_text:
    text5 += i

In [52]:
%%time
sentences5 = tokenize_text(text5)

Wall time: 23.5 s


In [53]:
print(sentences5[:5])

[['ch', 'ei', 'volere', 'udir', 'invan', 'pregai', 'ogni', 'adito', 'essere', 'chiudere', 'deriso', 'solo', 'partire', 'sapea', 'oggi', 'gioia', 'rammentare', 'alfine'], ['pentire', 'dicea', 'rivedere', 'condottier', 'de', 'nemico', 'ingrato'], ['bei', 'versovi', 'comprendere', 'voce', 'pubblico', 'riferire', 'opinione', 'assolutamente', 'falso'], ['sicchè', 'avere', 'mai', 'sapere', 'nulla', 'finora', 'fidanzamento'], ['mai', 've', 'assicuro']]


In [54]:
len(sentences5)

200080

In [55]:
%%time
w2v5 = Word2Vec(sentences=sentences5,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 3min 47s


In [56]:
w2v5.wv.most_similar(positive=['terrore'], topn=10)

[('sgomento', 0.5896665453910828),
 ('spavento', 0.5500238537788391),
 ('incutere', 0.5356026291847229),
 ('atterrire', 0.525767982006073),
 ('agonia', 0.5236062407493591),
 ('raccapriccio', 0.5233415365219116),
 ('rammarichío', 0.5171985626220703),
 ('rimorso', 0.5131546258926392),
 ('fremito', 0.5111005306243896),
 ('orrore', 0.5102239847183228)]

In [57]:
w2v5.save(os.path.join('../trained_models/Word2Vec0', 'w2v5.model'))

### Zeitraum 6: 1876-1900

In [58]:
text6 = ''

for i in df6.lemmatized_text:
    text6 += i

In [59]:
%%time
sentences6 = tokenize_text(text6)

Wall time: 27.6 s


In [60]:
print(sentences6[:5])

[['notte', 'intiera', 'intiera', 'battere', 'dente', 'nota', 'cicogna', 'crepare', 'fame', 'scoppiare', 'sete', 'schiattare', 'paura'], ['capitano', 'fare', 'sciogliere', 'rifocillare', 'meglio', 'procedere', 'interrogatorio'], ['sebbene', 'avere', 'contrattare', 'certo', 'modo', 'obbligo', 'morale', 'provvedere', 'sempre'], ['no', 'vecchierella', 'andare', 'nè', 'domani', 'nè', 'mai', 'abbandonare', 'molto', 'molto'], ['già', 'do', 'licenza', 'partire', 'dovere', 'anco', 'costare', 'grande', 'cattivoscarabocchiare', 'muro', 'accanto', 'frasca', 'canonico', 'motto', 'quando', 'cantare', 'credito', 'fare', 'oggi', 'no', 'domani', 'sì', 'patti', 'chiaro', 'amico', 'caro']]


In [61]:
len(sentences6)

253332

In [62]:
%%time
w2v6 = Word2Vec(sentences=sentences6,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 3min 22s


In [63]:
w2v6.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.5120231509208679),
 ('sgomento', 0.49296483397483826),
 ('calamità', 0.4630604386329651),
 ('carneficina', 0.4507082998752594),
 ('assalire', 0.4444493055343628),
 ('orrore', 0.4390488266944885),
 ('superstizioso', 0.4352823793888092),
 ('atroce', 0.43413981795310974),
 ('sbigottimento', 0.43274128437042236),
 ('ildi', 0.43120402097702026)]

In [64]:
w2v6.save(os.path.join('../trained_models/Word2Vec0', 'w2v6.model'))

### Zeitraum 7: 1901-1925

In [65]:
text7 = ''

for i in df7.lemmatized_text:
    text7 += i

In [66]:
%%time
sentences7 = tokenize_text(text7)

Wall time: 18 s


In [67]:
print(sentences7[:5])

[['preghiera', 'porre', 'letto'], ['mattino', 'vegnente', 'tempissimo', 'avere', 'riga', 'caro', 'quando', 'ieri', 'k', 'nigswinter', 'essere', 'rimanere', 'solo', 'te', 'avere', 'volere', 'dire', 'avere', 'potere', 'avere', 'vistare', 'avere', 'solamente', 'poterelampeggiare', 'gioia', 'quando', 'parlare', 'poesia', 'scrivere', 'notte'], ['ecco', 'esclamare', 'speranza', 'ieri'], ['quando', 'vedere', 'com', 'scrivere', 'cinque', 'verso', 'inesprimibile', 'amore', 'dire', 'guardare', 'fiso', 'stessoparte', 'almea'], ['sorseggiare', 'qualche', 'poco', 'poi', 'uscire', 'compiere', 'difficile', 'missione']]


In [68]:
len(sentences7)

178123

In [69]:
%%time
w2v7 = Word2Vec(sentences=sentences7,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 10s


In [70]:
w2v7.wv.most_similar(positive=['terrore'], topn=10)

[('sgomento', 0.5956056714057922),
 ('orrore', 0.5933123826980591),
 ('invincibile', 0.568263590335846),
 ('raccapriccio', 0.5543478727340698),
 ('ribrezzo', 0.5444951057434082),
 ('giovanezza', 0.5438083410263062),
 ('furore', 0.538895845413208),
 ('doloroso', 0.5327672958374023),
 ('gelare', 0.5319389700889587),
 ('estrema', 0.5288732051849365)]

In [71]:
w2v7.save(os.path.join('../trained_models/Word2vec0', 'w2v7.model'))

### Zeitraum 8: 1926-1950

In [72]:
text8 = ''

for i in df8.lemmatized_text:
    text8 += i

In [73]:
%%time
sentences8 = tokenize_text(text8)

Wall time: 14.8 s


In [74]:
print(sentences8[:5])

[['offesa', 'essere', 'principalmente', 'anzi', 'unicamente', 'incolpevole', 'quali', 'essere', 'umano', 'schierare', 'capo', 'gregario', 'sciopero', 'partito', 'volere', 'esso', 'esaltare'], ['così', 'oggi', 'dopo', 'due', 'lungo', 'sciopero', 'breve', 'distanza', 'altro', 'esempio', 'insigne', 'immedicata', 'follia', 'nazione', 'essere', 'socialista', 'volere', 'chiamare', 'giudice', 'moderatore', 'lotta', 'vita', 'nazionale', 'essere', 'avvelenare', 'tormentato'], ['monito', 'essere', 'formidabilmente', 'chiaro'], ['cronaca', 'sciopero', 'levare', 'verso', 'governo', 'essere', 'd', 'gravità', 'minore'], ['governo', 'avere', 'ereditare', 'tradizione', 'dubbiezza', 'quindi', 'debolezza', 'essere', 'certo', 'difficile', 'liberare', 'bisognare', 'pure', 'ogni', 'costo', 'liberare']]


In [75]:
len(sentences8)

120296

In [76]:
%%time
w2v8 = Word2Vec(sentences=sentences8,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 51s


In [77]:
w2v8.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.659649670124054),
 ('rabbico', 0.6314923167228699),
 ('invasare', 0.6233468651771545),
 ('virus', 0.610973596572876),
 ('rabbia', 0.6052799224853516),
 ('sgomento', 0.6034482717514038),
 ('stupore', 0.6022142767906189),
 ('scossa', 0.5934126973152161),
 ('subitaneo', 0.5883880257606506),
 ('tortura', 0.5856361389160156)]

In [78]:
w2v8.save(os.path.join('../trained_models/Word2Vec0', 'w2v8.model'))

### Zeitraum 9: 1951-1975

In [79]:
text9 = ''

for i in df9.lemmatized_text:
    text9 += i

In [80]:
%%time
sentences9 = tokenize_text(text9)

Wall time: 10.2 s


In [81]:
print(sentences9[:5])

[['caro', 'direttore', 'avere', 'leggere', 'ultimo', 'numero', 'rinascita', 'articolo', 'nilde', 'jotti', 'questione', 'fumetto', 'desiderare', 'esprimere', 'opinione', 'dire', 'subito', 'articolo', 'jotti', 'convincere'], ['esso', 'prendere', 'spunto', 'dibattito', 'corso', 'camera', 'stampa', 'ragazzo', 'giustamente', 'respingere', 'reazionario', 'inefficace', 'legge', 'proporre', 'democristiano', 'soltanto', 'contrario', 'principio', 'costituzionale', 'libertà', 'stampa', 'aut', 'decadenza', 'corruzione', 'delinquenza', 'giovane', 'dilagare', 'fumetto', 'essere', 'fatto', 'collegati', 'effetto', 'causa', 'bensì', 'manifestazione', 'diverso', 'realtà', 'unico'], ['bisognare', 'affrontare', 'risolvere', 'dire', 'giustamente', 'jotti', 'questione', 'orientamento', 'ideale', 'pratico', 'educazione', 'sviluppo', 'intellettuale', 'monile', 'giovane'], ['fare', 'mettere', 'dito', 'piaga', 'essere', 'ordine', 'economico', 'sociale', 'politico'], ['posizione', 'confronto', 'legge', 'fumetto'

In [82]:
len(sentences9)

82070

In [83]:
%%time
w2v9 = Word2Vec(sentences=sentences9,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 22s


In [84]:
w2v9.wv.most_similar(positive=['terrore'], topn=10)

[('maledizione', 0.9087971448898315),
 ('orrore', 0.9022539854049683),
 ('feroce', 0.8848339319229126),
 ('impotente', 0.8827385306358337),
 ('disperazione', 0.8815580606460571),
 ('tremendo', 0.8806217908859253),
 ('sconvolgere', 0.8744029998779297),
 ('mortale', 0.8607144951820374),
 ('orrendo', 0.8597286939620972),
 ('silvana', 0.8518133759498596)]

In [85]:
w2v9.save(os.path.join('../trained_models/Word2vec0', 'w2v9.model'))

### Zeitraum 10: 1976-2000

In [86]:
text10 = ''

for i in df10.lemmatized_text:
    text10+= i

In [87]:
%%time
sentences10 = tokenize_text(text10)

Wall time: 26 s


In [88]:
print(sentences10[:5])

[['essere', 'dire', 'andare', 'piano'], ['problema', 'affrontare', 'volta'], ['visti', 'così', 'blocco', 'spaventare'], ['ex', 'dc', 'riaffacciare', 'viminale', 'avere', 'governare', 'anno'], ['essere', 'pure', 'napoletano', 'usare', 'trebbiano']]


In [89]:
len(sentences10)

221424

In [90]:
%%time
w2v10 = Word2Vec(sentences=sentences10,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 3min 27s


In [91]:
w2v10.wv.most_similar(positive=['terrore'], topn=10)

[('pol', 0.5746635794639587),
 ('staliniano', 0.5730768442153931),
 ('incubo', 0.5587969422340393),
 ('terrorista', 0.5567649602890015),
 ('stalin', 0.5565236210823059),
 ('khmer', 0.5512040853500366),
 ('deportazione', 0.5476601719856262),
 ('sanguinario', 0.546563446521759),
 ('mietere', 0.5461190342903137),
 ('pot', 0.5460801720619202)]

In [92]:
w2v10.save(os.path.join('../trained_models/Word2Vec0', 'w2v10.model'))

### Zeitraum 11: 2001-2010

In [93]:
text11 = ''

for i in df11.lemmatized_text:
    text11+= i

In [94]:
%%time
sentences11 = tokenize_text(text11)

Wall time: 21.5 s


In [95]:
print(sentences11[:5])

[['durante', 'perquisizione', 'casa', 'due', 'malvivente', 'essere', 'stare', 'rinvenire', 'pistola', 'beretta', 'calibro', 'sw', 'regolarmente', 'denunciare', 'danilo', 'fazio', 'guardia', 'particolare', 'giurata', 'servizio', 'presso', 'agenzia', 'lodi', 'tfr', 'genitore', 'aiutare', 'figlio', 'invece', 'così', 'avere', 'aiutare', 'stato', 'centrale', 'fiat', 'avere', 'annunciare', 'arrivo', 'nuovo', 'limited', 'edition', 'denominata', 'blackjack', 'estate', 'italiano', 'cantare', 'insieme', 'bennato'], ['laureare', 'filosofia'], ['battista', 'bertoglio', 'nato', 'crevacuore', 'oggi', 'biella', 'dicembre', 'decedere', 'crevacuore', 'maggio', 'ristoratore', 'pioniere', 'socialismo'], ['membromundial', 'oggi', 'avere', 'esordire', 'commento', 'corrirere', 'sera'], ['dire']]


In [96]:
len(sentences11)

110514

In [97]:
%%time
w2v11 = Word2Vec(sentences=sentences11,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 4min 20s


In [98]:
w2v11.wv.most_similar(positive=['terrore'], topn=10)

[('terrorismo', 0.5717010498046875),
 ('oppressione', 0.5685752630233765),
 ('barbarie', 0.5531708598136902),
 ('insinuare', 0.5527194738388062),
 ('orrore', 0.5459117293357849),
 ('malvagità', 0.5454167723655701),
 ('ceceno', 0.5444887280464172),
 ('menzogna', 0.5441189408302307),
 ('carestia', 0.5426889061927795),
 ('fondamentalista', 0.5409315824508667)]

In [99]:
w2v11.save(os.path.join('../trained_models/Word2Vec0', 'w2v11.model'))

### Zeitraum 12: 2011-2016

In [100]:
text12 = ''

for i in df12.lemmatized_text:
    text12+= i

In [101]:
%%time
sentences12 = tokenize_text(text12)

Wall time: 20.5 s


In [102]:
print(sentences12[:5])

[['terzo', 'coppia', 'schiacciatore', 'essere', 'lanza', 'kovar', 'bei', 'tempo', 'quando', 'scuola', 'insegnare', 'economia', 'domestico', 'comportamentale', 'avere', 'fare', 'ottimo', 'partita', 'volta', 'essere', 'stare', 'po', 'sfortunato', 'avere', 'quasi', 'sempre', 'creare', 'tanto', 'occasione', 'cicloamatore', 'com', 'volere', 'essere', 'piccolo', 'social', 'network', 'punto', 'incontro', 'amante', 'bicicletta', 'giorno', 'potere', 'dichiarare', 'molto', 'globale', 'totale', 'valere', 'dire', 'intelligente', 'lucido', 'ambizioso', 'solidale'], ['essere', 'molto', 'semplice', 'bastare', 'cliccare', 'cromimi', 'poi', 'stare', 'scheda', 'profilo', 'infine', 'nome'], ['apparire', 'così', 'ilil', 'presidente', 'pietracupa', 'avere', 'preferire', 'spostare', 'altro', 'data', 'convocazione', 'consiglio', 'gay', 'potere', 'entrare', 'forza', 'armato', 'usa', 'senza', 'obbligo', 'tenere', 'nascondere', 'omosessualità', 'entrambi', 'essere', 'erbivoro', 'avere', 'corno', 'elaborare', 'a

In [103]:
len(sentences12)

71000

In [104]:
%%time
w2v12 = Word2Vec(sentences=sentences12,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 4min 31s


In [105]:
w2v12.wv.most_similar(positive=['terrore'], topn=10)

[('ripiombare', 0.6244481801986694),
 ('raccapriccio', 0.5971750617027283),
 ('incutere', 0.5971723794937134),
 ('desolazione', 0.5969581604003906),
 ('paura', 0.5870728492736816),
 ('angoscia', 0.5868802070617676),
 ('seminare', 0.5838741064071655),
 ('poliziesco', 0.5799481272697449),
 ('miseria', 0.5793553590774536),
 ('ferocia', 0.5748189687728882)]

In [106]:
w2v12.save(os.path.join('../trained_models/Word2Vec0', 'w2v12.model'))

### Zeitraum 13: 2017-2021

In [107]:
text13 = ''

for i in df13.lemmatized_text:
    text13+= i

In [108]:
%%time
sentences13 = tokenize_text(text13)

Wall time: 18.9 s


In [109]:
print(sentences13[:3])

[['candidato', 'dovere', 'essere', 'centrosinistra', 'indeciso', 'abbandonare', 'chávez', 'votare', 'qualcuno', 'destra', 'senso', 'egli', 'essere', 'sicuro', 'fanciulla', 'essere', 'altezza', 'cotanto', 'cliente', 'avere', 'personalmente', 'sperimentare', 'piano', 'orizzontale', 'direzione', 'essere', 'articolato', 'servizio', 'divisione', 'due', 'ufficio', 'seguito', 'elencare', 'svolgere', 'attività', 'principale', 'evidenziare', 'avere', 'fare', 'eco', 'cassidy', 'nasa', 'sottolineare', 'stare', 'applicare', 'stesso', 'precauzione', 'prendere', 'resto', 'mondo', 'distanziamento', 'sociale', 'post', 'profilo', 'facebook', 'poi', 'rimuovere', 'avere', 'chiedere', 'potere', 'avere', 'bambino', 'parotite', 'residente', 'marche', 'sottoporre', 'meglio', 'precisato', 'esperimento', 'essere', 'molto', 'medico'], ['nuovo', 'presidente', 'ordine', 'medici', 'indicazione', 'ats', 'insubria', 'controllare', 'ospite', 'dovere', 'avere', 'febbre', 'particolare', 'resistenza', 'antibiotico', 'sp

In [110]:
len(sentences13)

60249

In [111]:
%%time
w2v13 = Word2Vec(sentences=sentences13,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 4min 49s


In [112]:
w2v13.wv.most_similar(positive=['terrore'], topn=10)

[('incutere', 0.6577876210212708),
 ('orrore', 0.6476606726646423),
 ('crudeltà', 0.6427705883979797),
 ('mostruoso', 0.6361666917800903),
 ('seminare', 0.6358979940414429),
 ('disorientamento', 0.6341649889945984),
 ('esorcizzare', 0.6223713159561157),
 ('impotenza', 0.6199026703834534),
 ('lovecraft', 0.6192747950553894),
 ('oppressione', 0.6156490445137024)]

In [113]:
w2v13.save(os.path.join('../trained_models/Word2Vec0', 'w2v13.model'))